In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer


In [2]:
import numpy
import pandas
import scipy
import matplotlib
print("NumPy version:", numpy.__version__)
print("All imports successful!")

NumPy version: 1.26.4
All imports successful!


In [3]:
import matplotlib as plt

In [4]:
import matplotlib
print(matplotlib.__version__)


3.9.2


Dropping irrelevant columns that don't contribute to our Problem Statement or are completely empty

In [3]:
import pandas as pd

file_path = 'usecase_3_.csv'
data = pd.read_csv(file_path)


In [4]:
print(data.shape)

(257577, 32)


In [5]:
total_nulls = data.isnull().sum().sum()
print(f"Total number of null values: {total_nulls}")

Total number of null values: 1346212


In [6]:
data.nunique()

Unnamed: 0.1                  257577
Unnamed: 0                    257577
NCT Number                    257577
Study Title                   256885
Study URL                     257577
Acronym                        52454
Study Status                       4
Brief Summary                 256391
Study Results                      2
Conditions                    102973
Interventions                 198548
Primary Outcome Measures      244191
Secondary Outcome Measures    184792
Other Outcome Measures         18199
Sponsor                        26239
Collaborators                  35926
Sex                                3
Age                                6
Phases                             7
Enrollment                      6513
Funder Type                        9
Study Type                         2
Study Design                    1443
Other IDs                     254188
Start Date                      8049
Primary Completion Date         6925
Completion Date                 6981
F

In [7]:
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1','Study URL','Acronym','Sponsor','Age','Other IDs','Primary Completion Date','First Posted','Last Update Posted','Locations','Study Documents'], errors='ignore')

In [8]:
data = data.drop(columns=['Results First Posted'])

In [9]:
data.to_csv("usecase3_drop.csv")

In [12]:
import pandas as pd

file_path = 'drop_withdrawals.txt'
df = pd.read_csv(file_path, sep = "|")

In [13]:
print(df.shape)

(487399, 10)


In [14]:
df.head(10)

,id,nct_id,result_group_id,ctgov_group_code,period,reason,count,drop_withdraw_comment,reason_comment,count_units
0,5991139,NCT00827372,9120733,FG000,Overall Study,Adverse Event,5,NaN,NaN,NaN
1,5991140,NCT00827372,9120733,FG000,Overall Study,Lack of Efficacy,3,NaN,NaN,NaN
2,5991141,NCT01982760,9120734,FG000,Overall Study,Physician Decision,1,NaN,NaN,NaN
3,5991142,NCT01982760,9120735,FG001,Overall Study,Physician Decision,0,NaN,NaN,NaN
4,5991143,NCT01056276,9120736,FG000,Treatment Period-Cycles 1-8,Toxicity,2,NaN,NaN,NaN
5,5991144,NCT01056276,9120737,FG001,Treatment Period-Cycles 1-8,Toxicity,1,NaN,NaN,NaN
6,5991145,NCT01056276,9120736,FG000,Treatment Period-Cycles 1-8,Withdrawal by Subject,1,NaN,NaN,NaN
7,5991146,NCT01056276,9120737,FG001,Treatment Period-Cycles 1-8,Withdrawal by Subject,2,NaN,NaN,NaN
8,5991147,NCT01056276,9120736,FG000,Treatment Period-Cycles 1-8,Disease Progression,2,NaN,NaN,NaN
9,5991148,NCT01056276,9120737,FG001,Treatment Period-Cycles 1-8,Disease Progression,2,NaN,NaN,NaN


In [14]:
df = df.drop(columns=['id','result_group_id','ctgov_group_code','drop_withdraw_comment','reason_comment','count_units'])

In [18]:
import pandas as pd

file_path = 'facilities.txt'
df1 = pd.read_csv(file_path, sep = "|")

In [21]:
print(df1.shape)

(3085464, 5)


In [20]:
# we will be using the country column from this dataset - facilities
df1 = df1.drop(columns=['id','city','zip'])

In [22]:
df1.to_csv('facilities_drop.txt', index=False, sep='|')

In [1]:
import pandas as pd

file_path = 'reported_events.txt'
df2 = pd.read_csv(file_path, sep = "|")

In [2]:
print(df2.shape)

(9284467, 17)


In [5]:
df2 = df2.drop(columns=['id','result_group_id','ctgov_group_code','default_vocab','default_assessment'])

In [6]:
df2.to_csv('reported_events_drop.txt', index=False, sep='|')

In [7]:
import pandas as pd

file_path = 'eligibilities.txt'
df3 = pd.read_csv(file_path, sep = "|")

In [8]:
print(df3.shape)

(502301, 14)


In [9]:
df3 = df3.drop(columns=['id','gender','gender_based'])

In [10]:
df3.to_csv('eligibilities_drop.txt', index=False, sep='|')

Testing samples - what works best for our merged data

In [29]:
result = data[data['nct_id'] == 'NCT00827372']
print(result[['nct_id', 'reason_drop_withdrawals']])


             nct_id          reason_drop_withdrawals
114242  NCT00827372  Adverse Event; Lack of Efficacy


In [31]:
result1 = data[data['nct_id'] == 'NCT03175367']
print(result1[['reason_drop_withdrawals']])


                              reason_drop_withdrawals
62  Adverse Event; Lost to Follow-up; Noncomplianc...


Test examples to see what works best

In [34]:
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

data = pd.DataFrame({
    'nct_id': ['NCT00827372', 'NCT03175367'],
    'reason_drop_withdrawals': [
        'Adverse Event; Lack of Efficacy',
        'Adverse Event; Lost to Follow-up; Noncompliance with protocol; Physician Decision; Protocol Violation; Withdrawal by Subject'
    ]
})

data['reason_drop_withdrawals'] = data['reason_drop_withdrawals'].fillna('')

reasons_split = data['reason_drop_withdrawals'].str.split(';')

mlb = MultiLabelBinarizer()
reasons_encoded = mlb.fit_transform(reasons_split)

reason_df = pd.DataFrame(reasons_encoded, columns=mlb.classes_)

training_data = pd.concat([data, reason_df], axis=1)

print(training_data)


        nct_id                            reason_drop_withdrawals  \
0  NCT00827372                    Adverse Event; Lack of Efficacy   
1  NCT03175367  Adverse Event; Lost to Follow-up; Noncomplianc...   

    Lack of Efficacy   Lost to Follow-up   Noncompliance with protocol  \
0                  1                   0                             0   
1                  0                   1                             1   

    Physician Decision   Protocol Violation   Withdrawal by Subject  \
0                    0                    0                       0   
1                    1                    1                       1   

   Adverse Event  
0              1  
1              1  


The data was merged in {1_merge_files.ipynb} file

In [31]:
import pandas as pd
data = pd.read_csv("pakka_final.csv") # properly merged data

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\1284873162.py:2: DtypeWarning: Columns (0,13,19,20,21,22,23,24,26,27,28,36,37,38,39,40,42,46,203,204,206,207,208,209,210,211,212,213,214,225,226,227,229,232) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("pakka_final.csv")


In [33]:
data

,Unnamed: 0,NCT Number,Study Title,Study Status,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,...,reason_personal_reasons,reason_ongoing,reason_serious_adverse_event,reason_withdrawal_of_consent,reason_progressive_disease,reason_other_than_specified,reason_abnormal_laboratory_value(s),reason_missing,reason_unclassified,Unnamed: 285
0,0,NCT00559130,Efficacy Study of CytoSorb Hemoperfusion Devic...,COMPLETED,The hypothesis of this study is use of CytoSor...,NO,Acute Respiratory Distress Syndrome|Acute Lung...,DEVICE: CytoSorb Hemoperfusion,Relative IL-6 levels as a percent (%) of basel...,"Ventilator Free Days, Reduction cytokines TNF-...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1,NCT00937664,Safety and Tolerability Study of AZD7762 in Co...,NON COMPLETED,The primary purpose of this study is to find o...,NO,Cancer|Solid Tumors|Advanced Solid Malignancies,DRUG: AZD7762|DRUG: gemcitabine,Assessment of adverse events (based on CTCAE v...,Pharmacokinetic effect of AZD7762 when adminis...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2,NCT00441597,Does Atorvastatin Reduce Ischemia-Reperfusion ...,COMPLETED,To study the impact of 3 day exposure to atorv...,NO,Ischemia Reperfusion Injury|Cardiovascular Dis...,DRUG: atorvastatin,Annexin A 5 targeting in the non dominant then...,"workload during ischemic exercise, workload du...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,3,NCT03296228,Comparison of Dynamic Radiographs in Determini...,COMPLETED,The purpose of this study is to identify the f...,NO,Adolescent Idiopathic Scoliosis,"RADIATION: Flexibility Radiographs (supine, su...",Investigate the flexibility equivalence of dif...,Incorporate these findings into the Lenke Clas...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,4,NCT00421603,A Placebo-Controlled Study of Mixed Amphetamin...,COMPLETED,"The proposed protocol is a double-blind, place...",YES,Cocaine Dependence,DRUG: Adderall-XR and Topiramate|DRUG: Placebo,Three Weeks of Continuous Cocaine Abstinence a...,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258032,257572,NCT02360800,Trial Comparing Tisseel as an Adjunct to Tradi...,COMPLETED,Bleeding after redo cardiac surgery is a commo...,NO,Hemorrhage,PROCEDURE: Spray Fibrin Sealant|PROCEDURE: Tra...,Bleeding (Amount of blood (ml) collected in ch...,Allogeneic blood Sparing (Amount of Allogeneic...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
258033,257573,NCT02352506,Serum MIF in Acute Kidney Injury,COMPLETED,Acute kidney injury (AKI) is a common complica...,NO,Acute Kidney Injury,OTHER: Blood draw for measurement of MIF,"Macrophage migration inhibitory factor, Up to ...","Serum creatinine, Up to 14 days",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
258034,257574,NCT04996381,Feasibility of AI-based Heart Function Predict...,COMPLETED,The investigators will develop an artificial i...,NO,Chest X-ray for Clinical Evaluation,DIAGNOSTIC_TEST: Scanning Chest X-rays and per...,"Left Ventricular Ejection Fraction < 40%, Eval...",NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
258035,257575,NCT00380640,The Efficacy of Trimethoprim in Wound Healing ...,COMPLETED,The purpose of this study is to assess the eff...,NO,Epidermolysis Bullosa,DRUG: Trimethoprim|DRUG: Trimethoprim,Percentage change of area of the wound from vi...,"Total number of blisters at each visit, At 2 m...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [35]:
print(data.head())

  Unnamed: 0   NCT Number                                        Study Title  \
0          0  NCT00559130  Efficacy Study of CytoSorb Hemoperfusion Devic...   
1          1  NCT00937664  Safety and Tolerability Study of AZD7762 in Co...   
2          2  NCT00441597  Does Atorvastatin Reduce Ischemia-Reperfusion ...   
3          3  NCT03296228  Comparison of Dynamic Radiographs in Determini...   
4          4  NCT00421603  A Placebo-Controlled Study of Mixed Amphetamin...   

    Study Status                                      Brief Summary  \
0      COMPLETED  The hypothesis of this study is use of CytoSor...   
1  NON COMPLETED  The primary purpose of this study is to find o...   
2      COMPLETED  To study the impact of 3 day exposure to atorv...   
3      COMPLETED  The purpose of this study is to identify the f...   
4      COMPLETED  The proposed protocol is a double-blind, place...   

  Study Results                                         Conditions  \
0            NO  Acute

In [36]:
print(data.columns)

Index(['Unnamed: 0', 'NCT Number', 'Study Title', 'Study Status',
       'Brief Summary', 'Study Results', 'Conditions', 'Interventions',
       'Primary Outcome Measures', 'Secondary Outcome Measures',
       ...
       'reason_personal_reasons', 'reason_ongoing',
       'reason_serious_adverse_event', 'reason_withdrawal_of_consent',
       'reason_progressive_disease', 'reason_other_than_specified',
       'reason_abnormal_laboratory_value(s)', 'reason_missing',
       'reason_unclassified', 'Unnamed: 285'],
      dtype='object', length=286)


In [37]:
print(data.shape)

(258037, 286)


In [38]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258037 entries, 0 to 258036
Columns: 286 entries, Unnamed: 0 to Unnamed: 285
dtypes: float64(224), object(62)
memory usage: 563.0+ MB
None


In [39]:
print(data.isnull().sum())

Unnamed: 0                                232
NCT Number                                359
Study Title                               277
Study Status                              266
Brief Summary                             256
                                        ...  
reason_other_than_specified               690
reason_abnormal_laboratory_value(s)       693
reason_missing                            693
reason_unclassified                       693
Unnamed: 285                           258036
Length: 286, dtype: int64


In [40]:
data.drop(["Unnamed: 0", "Unnamed: 285"], axis =1 , inplace = True)

In [41]:
print(data.describe())

       country_Poland  country_Hungary  country_Turkey  country_Belgium  \
count   257570.000000    257571.000000   257571.000000    257571.000000   
mean         0.021210         0.012517        0.032713         0.029258   
std          0.144083         0.111177        0.177886         0.168529   
min          0.000000         0.000000        0.000000         0.000000   
25%          0.000000         0.000000        0.000000         0.000000   
50%          0.000000         0.000000        0.000000         0.000000   
75%          0.000000         0.000000        0.000000         0.000000   
max          1.000000         1.000000        1.000000         1.000000   

       country_Ukraine  country_Taiwan  \
count    257571.000000   257570.000000   
mean          0.006468        0.017890   
std           0.080164        0.132553   
min           0.000000        0.000000   
25%           0.000000        0.000000   
50%           0.000000        0.000000   
75%           0.000000        

Data Preprocessing

In [42]:
# Removing 'Years' and strip spaces

for col in ['maximum_age', 'minimum_age']:
    if col in data.columns:
        data[col] = data[col].str.replace('Years', '', regex=False).str.strip()
        data[col] = pd.to_numeric(data[col], errors='coerce')  

print(data[['maximum_age', 'minimum_age']].info())
print(data[['maximum_age', 'minimum_age']].head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258037 entries, 0 to 258036
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   maximum_age  131926 non-null  float64
 1   minimum_age  231557 non-null  float64
dtypes: float64(2)
memory usage: 3.9 MB
None
   maximum_age  minimum_age
0         80.0         18.0
1         74.0         20.0
2         50.0         18.0
3         18.0         10.0
4         60.0         18.0


In [43]:
binary_columns = ['adult', 'child', 'older_adult']

for col in binary_columns:
    data[col] = data[col].map({'t': 1, 'f': 0})

print(data[binary_columns].head())


   adult  child  older_adult
0    1.0    0.0          1.0
1    1.0    0.0          1.0
2    1.0    0.0          0.0
3    1.0    1.0          0.0
4    1.0    0.0          0.0


In [44]:
print(data['healthy_volunteers'].value_counts(dropna=False))

f           179625
t            72787
NaN           5408
0              141
1               75
80 Years         1
Name: healthy_volunteers, dtype: int64


In [45]:
import numpy as np

def map_healthy_volunteers(value):
    if value == 'f':
        return 0
    elif value == 't':
        return 1
    elif value == 0 or value == 1:  
        return value  
    elif pd.isna(value):
        return np.nan  
    else:
        return np.nan  

data['healthy_volunteers'] = data['healthy_volunteers'].apply(map_healthy_volunteers)

print(data['healthy_volunteers'].value_counts(dropna=False))


0.0    179625
1.0     72787
NaN      5625
Name: healthy_volunteers, dtype: int64


In [46]:
print(data['healthy_volunteers'].value_counts(dropna=False))

0.0    179625
1.0     72787
NaN      5625
Name: healthy_volunteers, dtype: int64


HANDLING MISSING VALUES

In [47]:
missing_values = data.isnull().sum() / len(data) * 100
print(missing_values.sort_values(ascending=False))

gender_description     98.468824
time_frame             88.329968
subjects_affected      85.167243
frequency_threshold    85.166856
subjects_at_risk       85.166856
                         ...    
Study Title             0.107349
Study Status            0.103086
Brief Summary           0.099211
Conditions              0.090685
Study Results           0.090685
Length: 284, dtype: float64


In [48]:
data = data.loc[:, missing_values < 90]
missing_values = data.isnull().sum() / len(data) * 100
print(missing_values.sort_values(ascending=False))

time_frame             88.329968
subjects_affected      85.167243
frequency_threshold    85.166856
subjects_at_risk       85.166856
population             80.638048
                         ...    
Study Title             0.107349
Study Status            0.103086
Brief Summary           0.099211
Conditions              0.090685
Study Results           0.090685
Length: 283, dtype: float64


Calculating missing value percentages for each group - COMPLETED / NON COMPLETED


In [49]:
def calculate_missing_values_by_group(group):
    return group.isnull().sum() / len(group) * 100

completed_group = data[data['Study Status'] == 'COMPLETED']
non_completed_group = data[data['Study Status'] != 'COMPLETED']

missing_completed = calculate_missing_values_by_group(completed_group)
missing_non_completed = calculate_missing_values_by_group(non_completed_group)

missing_comparison = pd.DataFrame({
    'COMPLETED (%)': missing_completed,
    'NON COMPLETED (%)': missing_non_completed
})

missing_comparison = missing_comparison.sort_values(by='COMPLETED (%)', ascending=False)

print(missing_comparison)


                     COMPLETED (%)  NON COMPLETED (%)
time_frame               88.508059          87.259138
subjects_affected        85.343651          84.106536
frequency_threshold      85.343199          84.106536
subjects_at_risk         85.343199          84.106536
population               79.768760          85.864927
...                            ...                ...
Study Status              0.000000           0.722924
Study Title               0.000000           0.752820
Study Results             0.000000           0.635956
Conditions                0.000000           0.635956
NCT Number                0.000000           0.975676

[283 rows x 2 columns]


In [50]:
print(data.describe())

       country_Poland  country_Hungary  country_Turkey  country_Belgium  \
count   257570.000000    257571.000000   257571.000000    257571.000000   
mean         0.021210         0.012517        0.032713         0.029258   
std          0.144083         0.111177        0.177886         0.168529   
min          0.000000         0.000000        0.000000         0.000000   
25%          0.000000         0.000000        0.000000         0.000000   
50%          0.000000         0.000000        0.000000         0.000000   
75%          0.000000         0.000000        0.000000         0.000000   
max          1.000000         1.000000        1.000000         1.000000   

       country_Ukraine  country_Taiwan  \
count    257571.000000   257570.000000   
mean          0.006468        0.017890   
std           0.080164        0.132553   
min           0.000000        0.000000   
25%           0.000000        0.000000   
50%           0.000000        0.000000   
75%           0.000000        

HANDLING NULL VALUES

In [51]:
print(data["minimum_age"].dtype)

float64


In [61]:
data["Enrollment"] = pd.to_numeric(data["Enrollment"], errors='coerce')


C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\1155625598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Enrollment"] = pd.to_numeric(data["Enrollment"], errors='coerce')


In [63]:
data["Enrollment"].dtype


dtype('float64')

In [64]:
data['subjects_affected'].dtype

dtype('O')

In [65]:
# changing from object dtype to float - column name : subjects_affected
data["subjects_affected"] = pd.to_numeric(data["subjects_affected"], errors='coerce')

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\1887721029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["subjects_affected"] = pd.to_numeric(data["subjects_affected"], errors='coerce')


In [67]:
data['subjects_affected']

0          NaN
1          NaN
2          NaN
3          NaN
4         17.0
          ... 
258032     NaN
258033     NaN
258034     NaN
258035     NaN
258036     NaN
Name: subjects_affected, Length: 258037, dtype: float64

In [72]:
data['maximum_age'].dtype

dtype('float64')

Numerical Columns Imputation

In [73]:
numerical_columns = [
    'Enrollment', 'subjects_affected', 'subjects_at_risk', 'frequency_threshold',
    'minimum_age', 'maximum_age']

In [75]:
for column in numerical_columns:
    median_value = data[column].median()
    data[column].fillna(median_value, inplace=True)

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\95743752.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column].fillna(median_value, inplace=True)


In [76]:
data['subjects_affected']

0          9.0
1          9.0
2          9.0
3          9.0
4         17.0
          ... 
258032     9.0
258033     9.0
258034     9.0
258035     9.0
258036     9.0
Name: subjects_affected, Length: 258037, dtype: float64

In [77]:
data["Enrollment"]

0         100.0
1          24.0
2          30.0
3         134.0
4          81.0
          ...  
258032    120.0
258033    500.0
258034    505.0
258035     10.0
258036    208.0
Name: Enrollment, Length: 258037, dtype: float64

Creating a duration column from the "Start Date" and "Completion Date" Columns

In [78]:
import pandas as pd
import numpy as np

def calculate_duration(data, start_col='Start Date', end_col='Completion Date'):
    """
    Calculate duration in months between start and completion dates, handling NaN values
    """
    data[start_col] = pd.to_datetime(data[start_col], errors='coerce')
    data[end_col] = pd.to_datetime(data[end_col], errors='coerce')
    
    def month_difference(start, end):
        if pd.isna(start) or pd.isna(end):
            return np.nan
        return abs((end.year - start.year) * 12 + (end.month - start.month))
    
    data['duration'] = data.apply(lambda row: month_difference(row[start_col], row[end_col]), axis=1)
    
    return data

data = calculate_duration(data)

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\2534147252.py:9: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data[start_col] = pd.to_datetime(data[start_col], errors='coerce')
C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\2534147252.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[start_col] = pd.to_datetime(data[start_col], errors='coerce')
C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\2534147252.py:10: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent p

In [79]:
data["duration"]

0         43.0
1         19.0
2         25.0
3         35.0
4         39.0
          ... 
258032    19.0
258033    96.0
258034     0.0
258035    12.0
258036     NaN
Name: duration, Length: 258037, dtype: float64

In [80]:
data["duration"].isna().sum()

9358

In [81]:
import numpy as np

numerical_columns = ["duration"]

for column in numerical_columns:
    median_value = data[column].median()
    data[column].fillna(median_value, inplace=True)

print(data[numerical_columns].isnull().sum())


duration    0
dtype: int64


In [82]:
data["duration"]

0         43.0
1         19.0
2         25.0
3         35.0
4         39.0
          ... 
258032    19.0
258033    96.0
258034     0.0
258035    12.0
258036    23.0
Name: duration, Length: 258037, dtype: float64

In [83]:
#healthy_volunteers: 0,1,-1
data['healthy_volunteers'] = data['healthy_volunteers'].fillna(-1)

In [84]:
print(data['healthy_volunteers'].value_counts(dropna=False))

 0.0    179625
 1.0     72787
-1.0      5625
Name: healthy_volunteers, dtype: int64


In [85]:
def calculate_missing_values_by_group(group):
    return group.isnull().sum() / len(group) * 100

completed_group = data[data['Study Status'] == 'COMPLETED']
non_completed_group = data[data['Study Status'] != 'COMPLETED']

missing_completed = calculate_missing_values_by_group(completed_group)
missing_non_completed = calculate_missing_values_by_group(non_completed_group)

missing_comparison = pd.DataFrame({
    'COMPLETED (%)': missing_completed,
    'NON COMPLETED (%)': missing_non_completed
})

missing_comparison = missing_comparison.sort_values(by='COMPLETED (%)', ascending=False)

print(missing_comparison)

                 COMPLETED (%)  NON COMPLETED (%)
time_frame           88.508059          87.259138
population           79.768760          85.864927
sampling_method      79.763788          85.875798
Collaborators        67.493514          67.585270
Phases               58.315781          44.038592
...                        ...                ...
Conditions            0.000000           0.635956
Study Results         0.000000           0.635956
Brief Summary         0.000000           0.695747
Study Status          0.000000           0.722924
duration              0.000000           0.000000

[284 rows x 2 columns]


Categorical Column Imputation : Filling null values with placeholders

In [86]:
data['sampling_method'].unique()

array([nan, 'NON_PROBABILITY_SAMPLE', 'PROBABILITY_SAMPLE', 't', 'f',
       'Withdrawal by Subject; Adverse Event; Lost to Follow-up; Protocol Violation; Lack of Efficacy; Physician Decision; Noncompliance; Cough Improvement',
       '1',
       "Inclusion Criteria:~1. Subject must be greater than or equal to 18 years of age.~2. Subjects must have an advanced hematologic malignancy including:~Phase 1/ Dose escalation:~1. Diagnosis of acute myelogenous leukemia (AML) according to World Health Organization (WHO) criteria;~   * Disease refractory or relapsed (defined as the reappearance of \\> 5% blasts in the bone marrow).~  * Untreated AML, greater than or equal to 60 years of age and are not candidates for standard therapy due to age, performance status, and/or adverse risk factors, according to the treating physician and with approval of the Medical Monitor;~2. Diagnosis of Myelodysplastic syndrome (MDS) according to WHO classification with refractory anemia with excess blasts (RAEB-

In [87]:
filtered_data = data[data['sampling_method'].isin([np.nan, 'NON_PROBABILITY_SAMPLE', 'PROBABILITY_SAMPLE'])]

In [88]:
filtered_data.shape

(257816, 284)

In [89]:
filtered_data['sampling_method'].unique()

array([nan, 'NON_PROBABILITY_SAMPLE', 'PROBABILITY_SAMPLE'], dtype=object)

In [90]:
#sampling_method - column name

filtered_data['sampling_method'] = filtered_data['sampling_method'].fillna('UNKNOWN_SAMPLE')

print(filtered_data['sampling_method'].isnull().sum())
print(filtered_data['sampling_method'].unique())


0
['UNKNOWN_SAMPLE' 'NON_PROBABILITY_SAMPLE' 'PROBABILITY_SAMPLE']


C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\1132756599.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['sampling_method'] = filtered_data['sampling_method'].fillna('UNKNOWN_SAMPLE')


In [91]:
filtered_data['sampling_method'].unique()

array(['UNKNOWN_SAMPLE', 'NON_PROBABILITY_SAMPLE', 'PROBABILITY_SAMPLE'],
      dtype=object)

In [92]:
filtered_data['Sex'].unique()

array(['ALL', 'MALE', 'FEMALE', nan, '30-09-2010',
       ' and renal function~* For women of childbearing potential: agreement to remain abstinent (refrain from heterosexual intercourse) or use a highly effective contraceptive method with a failure rate of \\<1% per year during the treatment period and for at least five elimination half-lives (24 weeks) after the last dose of emicizumab~Exclusion Criteria:~* Inherited or acquired bleeding disorder other than hemophilia A~* Ongoing (or plan to receive during the study) immune tolerance induction (ITI) therapy (prophylaxis regimens with FVIII and/or bypassing agents must be discontinued prior to enrollment). Patients receiving ITI therapy will be eligible following the completion of a 72-hour washout period prior to the first emicizumab administration~* History of illicit drug or alcohol abuse within 12 months prior to screening',
       '27-03-2009', '0', '09-12-2010', '02-06-2010', '09-02-2011'],
      dtype=object)

In [93]:
filtered_data1 = filtered_data[filtered_data['Sex'].isin([np.nan, 'ALL', 'MALE','FEMALE'])]

In [94]:
filtered_data1['Sex'].unique()

array(['ALL', 'MALE', 'FEMALE', nan], dtype=object)

In [95]:
filtered_data1.shape

(257809, 284)

In [96]:
#Sex - column name

filtered_data1['Sex'] = filtered_data1['Sex'].fillna('UNKNOWN_Sex')

print(filtered_data1['Sex'].isnull().sum())
print(filtered_data1['Sex'].unique())

0
['ALL' 'MALE' 'FEMALE' 'UNKNOWN_Sex']


C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\3665607310.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1['Sex'] = filtered_data1['Sex'].fillna('UNKNOWN_Sex')


In [97]:
filtered_data1['Phases'].unique()

array([nan, 'PHASE1', 'PHASE4', 'PHASE2', 'PHASE3', 'EARLY_PHASE1',
       'PHASE1|PHASE2', 'PHASE2|PHASE3'], dtype=object)

In [98]:
#Phases 

filtered_data1['Phases'] = filtered_data1['Phases'].fillna('UNKNOWN_PHASE')

print(f"Missing values in 'Phases': {filtered_data1['Phases'].isnull().sum()}")
print(f"Sample categories in 'Phases': {filtered_data1['Phases'].unique()[:10]}")


Missing values in 'Phases': 0
Sample categories in 'Phases': ['UNKNOWN_PHASE' 'PHASE1' 'PHASE4' 'PHASE2' 'PHASE3' 'EARLY_PHASE1'
 'PHASE1|PHASE2' 'PHASE2|PHASE3']


C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\687037914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1['Phases'] = filtered_data1['Phases'].fillna('UNKNOWN_PHASE')


In [99]:
filtered_data1['population'].unique()

array([nan,
       'The study will be divided into two phases. In Phase 1, an exploratory pilot study will be performed with up to 3 flexibility films done on the same patient in different spinal centres which are in Hong Kong and Turkey.',
       "The test population is pediatric patient (0-18 years of age) with known liver disease with plans to undergo a liver biopsy within 1 month of ultrasound exam. Underlying diagnoses include but are not limited to biliary atresia, congenital fibrosis-cholestasis, Alagille syndrome, Caroli's disease, choledochal cyst, alpha-1-antitrypsin deficiency, progressive familial intrahepatic cholestasis (PFIC), viral hepatitis, glycogenosis, fructosemia, Wilson disease, cystic fibrosis, autosomal recessive polycystic kidney disease (ARPCKD), mesenterico-caval shunt, post liver transplant, and nonalcoholic steatohepatitis (NASH).~The control population is any pediatric patient (0-18 years of age) undergoing an abdominal ultrasound for reasons other than li

In [100]:
#Population

filtered_data1['population'] = filtered_data1['population'].fillna('UNKNOWN_POPULATION')

print(f"Missing values in 'population': {filtered_data1['population'].isnull().sum()}")
print(f"Sample categories in 'population': {filtered_data1['population'].unique()[:10]}")


Missing values in 'population': 0
Sample categories in 'population': ['UNKNOWN_POPULATION'
 'The study will be divided into two phases. In Phase 1, an exploratory pilot study will be performed with up to 3 flexibility films done on the same patient in different spinal centres which are in Hong Kong and Turkey.'
 "The test population is pediatric patient (0-18 years of age) with known liver disease with plans to undergo a liver biopsy within 1 month of ultrasound exam. Underlying diagnoses include but are not limited to biliary atresia, congenital fibrosis-cholestasis, Alagille syndrome, Caroli's disease, choledochal cyst, alpha-1-antitrypsin deficiency, progressive familial intrahepatic cholestasis (PFIC), viral hepatitis, glycogenosis, fructosemia, Wilson disease, cystic fibrosis, autosomal recessive polycystic kidney disease (ARPCKD), mesenterico-caval shunt, post liver transplant, and nonalcoholic steatohepatitis (NASH).~The control population is any pediatric patient (0-18 years of

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\788675119.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1['population'] = filtered_data1['population'].fillna('UNKNOWN_POPULATION')


In [101]:
filtered_data1['Secondary Outcome Measures'].unique()

array(['Ventilator Free Days, Reduction cytokines TNF-α, IL-1b, IL-10, CRP, 28-day all cause mortality, Oxygen Index (OI), P/F ratios, MODS scores, 28 Days',
       'Pharmacokinetic effect of AZD7762 when administered alone and in combination with gemcitabine, According to protocol specified schedule, the number of PK samples collected during Cycle 0 and Cycle 1|Anti-tumor activity of AZD7762 when administered in combination with gemcitabine by assessment of Response Evaluation Criteria in Solid Tumors (RECIST), Every 2-3 cycles',
       'workload during ischemic exercise, workload during 10minutes of ischemic exercise|effect of 3-day treatment with atorvastatin 80mg daily on serum lipid levels, fasting lipid levels before and at first day after 3 day treatment with atorvastatin',
       ...,
       'Allogeneic blood Sparing (Amount of Allogeneic Blood units (number) transfused in the first 4 days after surgery), Amount of Allogeneic Blood units (number) transfused in the first 4 days 

In [102]:
#Secondary Outcome Measures

filtered_data1['Secondary Outcome Measures'] = filtered_data1['Secondary Outcome Measures'].fillna('UNKNOWN_MEASURES')

print(f"Missing values in 'Secondary Outcome Measures': {filtered_data1['Secondary Outcome Measures'].isnull().sum()}")
print(f"Sample categories in 'Secondary Outcome Measures': {filtered_data1['Secondary Outcome Measures'].unique()[:10]}")


C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\3610122958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1['Secondary Outcome Measures'] = filtered_data1['Secondary Outcome Measures'].fillna('UNKNOWN_MEASURES')


Missing values in 'Secondary Outcome Measures': 0
Sample categories in 'Secondary Outcome Measures': ['Ventilator Free Days, Reduction cytokines TNF-α, IL-1b, IL-10, CRP, 28-day all cause mortality, Oxygen Index (OI), P/F ratios, MODS scores, 28 Days'
 'Pharmacokinetic effect of AZD7762 when administered alone and in combination with gemcitabine, According to protocol specified schedule, the number of PK samples collected during Cycle 0 and Cycle 1|Anti-tumor activity of AZD7762 when administered in combination with gemcitabine by assessment of Response Evaluation Criteria in Solid Tumors (RECIST), Every 2-3 cycles'
 'workload during ischemic exercise, workload during 10minutes of ischemic exercise|effect of 3-day treatment with atorvastatin 80mg daily on serum lipid levels, fasting lipid levels before and at first day after 3 day treatment with atorvastatin'
 'Incorporate these findings into the Lenke Classification of AIS, To incorporate these findings into the Lenke Classification o

In [103]:
#Primary Outcome Measures

filtered_data1['Primary Outcome Measures'] = filtered_data1['Primary Outcome Measures'].fillna('UNKNOWN_PRIMARY_MEASURES')

print(f"Missing values in 'Secondary Outcome Measures': {filtered_data1['Primary Outcome Measures'].isnull().sum()}")
print(f"Sample categories in 'Secondary Outcome Measures': {filtered_data1['Primary Outcome Measures'].unique()[:10]}")


C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\3386266803.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1['Primary Outcome Measures'] = filtered_data1['Primary Outcome Measures'].fillna('UNKNOWN_PRIMARY_MEASURES')


Missing values in 'Secondary Outcome Measures': 0
Sample categories in 'Secondary Outcome Measures': ['Relative IL-6 levels as a percent (%) of baseline will be lower in subjects receiving CytoSorb treatment in conjunction with the standard of care as compared to control subjects receiving only the standard of care for ARDS/ALI in the setting of sepsis., 7 Days'
 'Assessment of adverse events (based on CTCAE version 3.0), laboratory values, vital sign measurements, cardiac markers, ECG, Echocardiogram, Laboratory values weekly for all treatment cycles, and the other variables weekly for Cycle 0-1, then every cycle (3-4 weeks) from Cycle 2'
 'Annexin A 5 targeting in the non dominant thenar muscle after ischemic exercise, as a indicator for ischemia reperfusion injury., 60 and 240 minutes after ischemic exercise'
 'Investigate the flexibility equivalence of different bending methods, and their predictability of the final outcome, To investigate the flexibility equivalence of different b

In [105]:
filtered_data1['adult'].unique()

array([ 1.,  0., nan])

In [106]:
filtered_data1.drop(["time_frame","Collaborators","Completion Date","Start Date"], axis =1 ,inplace = True)

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\1848144911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1.drop(["time_frame","Collaborators","Completion Date","Start Date"], axis =1 ,inplace = True)


In [108]:
filtered_data1["criteria"].unique()

array(['Inclusion Criteria:~* Signed informed consent document (ICD)~* Male or female ≥ 18 and ≤ 80 years of age.~* Subjects must have diagnosis of ARDS or ALI, based on ARDSNet Definition, established within last 72 hours, confirmed by clinical, radiological, or physiologic findings~* Subject must be intubated~* ≤ 3 days on a ventilator prior to enrollment~* Subjects must have confirmed diagnosis of sepsis~* Subject must have had at least 24 hours of antibiotic therapy~* Pre-menopausal female subjects must have negative pregnancy test.~* Subject must be available for periodic blood sampling, study related assessments, and management at the treating institution for the duration of the study. Subject must have permanent home address to allow completion of 60 day follow-up.~* Subject or health care proxy has the ability to understand and willingness to sign the informed consent form.~Exclusion Criteria:~* Currently participating in another clinical study involving investigational chemica

In [109]:
fill_values = {
    'Interventions': 'UNKNOWN_INTERVENTION',
    'criteria' : 'UNKNOWN_CRITERIA'
}

filtered_data1.fillna(value=fill_values, inplace=True)

missing_values_after = filtered_data1[list(fill_values.keys())].isnull().sum()
print("Missing values after handling:")
print(missing_values_after)

Missing values after handling:
Interventions    0
criteria         0
dtype: int64


C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\3141639914.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1.fillna(value=fill_values, inplace=True)


In [111]:
fill_values = {
    'Study Design': 'UNKNOWN_DESIGN',
    'Study Type': 'UNKNOWN_STUDY_TYPE',
    'Funder Type' : 'UNKNOWN_FUNDER'
}

filtered_data1.fillna(value=fill_values, inplace=True)

missing_values_after = filtered_data1[list(fill_values.keys())].isnull().sum()
print("Missing values after handling:")
print(missing_values_after)

Missing values after handling:
Study Design    0
Study Type      0
Funder Type     0
dtype: int64


C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\2958426727.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1.fillna(value=fill_values, inplace=True)


Handling Columns that are encoded : filling nan values with -1

In [112]:
columns_to_fill = ['older_adult', 'adult', 'child']
for col in columns_to_fill:
    filtered_data1[col] =filtered_data1[col].fillna(-1)

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\581757599.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1[col] =filtered_data1[col].fillna(-1)


In [113]:
# null values handling in columns starting with '_adverse_event_cluster_' : these are encoded columns

adverse_columns = [col for col in filtered_data1.columns if col.startswith('_adverse_event_cluster_')]
for col in adverse_columns:
   filtered_data1[col] = filtered_data1[col].fillna(-1)

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\3907111339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1[col] = filtered_data1[col].fillna(-1)


In [188]:
filtered_data1.to_csv(r"C:\Users\Siddhant Nijhawan\Downloads\Nest_Codes\Dropped_columns_files\filtered_data1.csv")

In [198]:
filtered_data1 = pd.read_csv(r"C:\Users\Siddhant Nijhawan\Downloads\Nest_Codes\Dropped_columns_files\filtered_data1.csv")

In [114]:
#starting with "reason_"

reason_columns = [col for col in filtered_data1.columns if col.startswith('reason_')]
for col in reason_columns:
   filtered_data1[col] = filtered_data1[col].fillna(-1)

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\1062458348.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1[col] = filtered_data1[col].fillna(-1)


In [115]:
#starting with "organ_system_"

organ_system_columns = [col for col in filtered_data1.columns if col.startswith('organ_system_')]
for col in organ_system_columns:
   filtered_data1[col] = filtered_data1[col].fillna(-1)

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\4161749727.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1[col] = filtered_data1[col].fillna(-1)


In [116]:
#starting with "period_"

prefixes = ['period_']
for prefix in prefixes:
   columns = [col for col in filtered_data1.columns if col.startswith(prefix)]
   for col in columns:
       filtered_data1[col] = filtered_data1[col].fillna(-1)

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\2214854627.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1[col] = filtered_data1[col].fillna(-1)


In [117]:
prefixes = ['assessment_type_', 'country_', 'event_type_']
for prefix in prefixes:
   columns = [col for col in filtered_data1.columns if col.startswith(prefix)]
   for col in columns:
       filtered_data1[col] = filtered_data1[col].fillna(-1)

C:\Users\yuvik\AppData\Local\Temp\ipykernel_28964\897850941.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data1[col] = filtered_data1[col].fillna(-1)


In [126]:
total_nulls = filtered_data3.isnull().sum().head(10)
print(f"Total number of null values: {total_nulls}")

Total number of null values: NCT Number                    0
Study Title                   0
Study Status                  0
Brief Summary                 0
Study Results                 0
Conditions                    0
Interventions                 0
Primary Outcome Measures      0
Secondary Outcome Measures    0
Sex                           0
dtype: int64


In [120]:
filtered_data2 = filtered_data1.dropna(subset=['NCT Number'])

In [121]:
filtered_data3 = filtered_data2.dropna(subset=['Study Title'])

In [124]:
total_nulls = filtered_data3.isnull().sum().sum()
print(f"Total number of null values: {total_nulls}")

Total number of null values: 0


In [123]:
filtered_data3.to_csv(r"null_values_dealt.csv")